# AVITO PROJECT

## Team Member

* Sandikha Rahardi - РИМ 130908

## Load Data

### Import Package

In [113]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree

### Set Dataframe

In [114]:
dataset_filename = "assets/avito_dataset.xlsx"
main_df = pd.read_excel(dataset_filename, sheet_name="Sheet1")

## Data Cleaning

### Important Column

In [115]:
# Define important column to predict the price
important_columns = [
    'Цена', 'Метро', 'Метро2', 'Метро. Время', 
    'Район', 'Продавец. Компания', 'Статус', 'Количество комнат',
    'Общая площадь, число', 'Этаж', 'Санузел', 
    'Балкон или лоджия', 'Окна', 'Ремонт', 
    'Тип дома', 'Этажей в доме', 'Площадь кухни, число'
]
print("Total original column : " , main_df.columns.__len__())
print("Total important column : " , important_columns.__len__())
main_df = main_df[important_columns]

Total original column :  71
Total important column :  17


In [116]:
print(main_df.shape)
main_df.isnull().sum() / main_df.shape[0] * 100

(19803, 17)


Цена                     0.000000
Метро                    0.489825
Метро2                  47.856385
Метро. Время            47.856385
Район                   52.310256
Продавец. Компания       0.000000
Статус                   0.000000
Количество комнат        0.000000
Общая площадь, число     0.000000
Этаж                     0.000000
Санузел                 40.301974
Балкон или лоджия       32.692016
Окна                    56.824724
Ремонт                  60.793819
Тип дома                13.750442
Этажей в доме            0.000000
Площадь кухни, число    19.491996
dtype: float64

### Clean the dataset

#### Remove duplicated dataset

In [117]:
print("Duplicated data : ", main_df.duplicated().sum())
main_df = main_df.drop_duplicates().reset_index(drop=True)
print("Duplicated data (now) : ", main_df.duplicated().sum())

Duplicated data :  276
Duplicated data (now) :  0


#### Clean about 'Метро'

In [118]:
main_df.dropna(subset=['Метро'], inplace=True)

def preprocess_metro_time(row):
    if(type(row) == str):
        row = row.replace("мин.","").strip()
        if(row.__contains__("от")):
            row = float(row.replace("от","").strip())
        elif(row.__contains__("до")):
            row = float(row.replace("до","").strip())
        elif(row.__contains__("–")):
            row = (int(row.split("–")[0]) + int(row.split("–")[1])) / 2
    return row

main_df['Метро. Время'] = main_df['Метро. Время'].apply(preprocess_metro_time)
mean_metro_time = round(pd.to_numeric(main_df['Метро. Время']).mean(), 1)
main_df['Метро. Время'].fillna(mean_metro_time, inplace=True)
main_df['Метро2'] = main_df['Метро2'].apply(lambda x: "Нет" if pd.isna(x) else "Есть")

#### Clean about 'Район'

In [119]:
main_df['Район'].fillna("Другой", inplace=True)

#### Clean about 'Количество комнат'

In [120]:
main_df['Количество комнат'] = main_df['Количество комнат'].apply(lambda x: 0 if x == 'Студия' else x)

max_room = pd.to_numeric(main_df['Количество комнат'], errors='coerce').max()
mean_room = pd.to_numeric(main_df['Количество комнат'], errors='coerce').mean()

main_df['Количество комнат'] = main_df['Количество комнат'].apply(lambda x: max_room if x == 'многокомнатные' else x)
main_df['Количество комнат'] = main_df['Количество комнат'].apply(lambda x: round(mean_room) if x == 'Своб. планировка' else x)

main_df['Количество комнат'].value_counts()

Количество комнат
2.0    6516
1.0    6276
3.0    3592
0.0    2470
4.0     492
5.0      67
6.0       9
7.0       8
Name: count, dtype: int64

> Note: Студия does not have room ( 0 room )

#### Clean about 'Общая площадь, число'

In [121]:
main_df['Общая площадь, число'] = main_df['Общая площадь, число'].apply(lambda x: float(x.replace(",", ".")) if type(x) == str else int(x))

#### Clean about 'Этаж'

In [122]:
# Clean dataset 'Этаж' of 'Этажей в доме'
main_df['Этаж'] = main_df['Этаж'].apply(lambda x: int(x.split("из")[0].strip()))

#### Clean about 'Санузел'

In [123]:
main_df['Санузел'].fillna('Другой', inplace=True)

#### Clean about 'Балкон или лоджия'

In [124]:
main_df['Балкон или лоджия'].fillna('Другой', inplace=True)

#### Clean about 'Окна'

In [125]:
main_df['Окна'].fillna('Другой', inplace=True)

#### Clean about 'Ремонт'

In [126]:
main_df['Ремонт'].fillna('Другой', inplace=True)

#### Clean about 'Тип дома'

In [127]:
main_df['Тип дома'].fillna('Другой', inplace=True)

#### Clean about 'Площадь кухни, число'

In [128]:
main_df['Площадь кухни, число'] = main_df['Площадь кухни, число'].apply(lambda x: float(x.replace(",", ".")) if type(x) == str else x)
print("Data Null :" , main_df['Площадь кухни, число'].isna().sum())
print("Data Mode :" , main_df['Площадь кухни, число'].mode()[0])
main_df['Площадь кухни, число'].describe()

Data Null : 3643
Data Mode : 6.0


count    15787.000000
mean        14.009001
std          7.042748
min          2.000000
25%          8.200000
50%         13.900000
75%         18.000000
max        100.000000
Name: Площадь кухни, число, dtype: float64

In [129]:
mode_number = round(main_df['Площадь кухни, число'].mode()[0], 1)
mean_number = round(main_df['Площадь кухни, число'].mean(), 1)
main_df['Площадь кухни, число'].fillna(np.random.randint(mode_number,mean_number+1), inplace=True)

### Split Dataset

In [130]:
# Split the data into training (80%), testing (15%), and validation (5%)
train_df, temp_df = train_test_split(main_df, test_size=0.2, random_state=42)
test_df, val_df = train_test_split(temp_df, test_size=0.25, random_state=42)
print(train_df.shape)
print(test_df.shape)
print(val_df.shape)

(15544, 17)
(2914, 17)
(972, 17)


### Divided into Categorical and Numerical dataset

In [131]:
label = 'Цена'
numerical_columns = ['Метро. Время', 'Количество комнат', 'Общая площадь, число', 'Этаж', 'Этажей в доме', 'Площадь кухни, число']
categorical_columns = ['Метро', 'Метро2', 'Район', 'Продавец. Компания', 'Статус', 'Санузел', 'Балкон или лоджия', 'Окна', 'Ремонт', 'Тип дома']

train_num_df = train_df[numerical_columns]
train_cat_df = train_df[categorical_columns]
test_num_df = test_df[numerical_columns]
test_cat_df = test_df[categorical_columns]
val_num_df = val_df[numerical_columns]
val_cat_df = val_df[categorical_columns]